<a href="https://colab.research.google.com/github/Shujaat123/DeepVAE-SRC/blob/main/AFP_SRC_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys, os, re, gc
import numpy as np
import pandas as pd
from random import sample

## Models
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Dropout
from keras import metrics
from keras import optimizers
from keras.utils.np_utils import to_categorical

import numpy.linalg as LA
from sklearn.model_selection import train_test_split

## Perfmetrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, matthews_corrcoef, balanced_accuracy_score, precision_recall_fscore_support
from sklearn.metrics import auc, average_precision_score, precision_recall_curve, roc_curve

## utilities
from matplotlib import pyplot as plt
!pip install wget
import wget

In [2]:
file1_path = 'https://raw.githubusercontent.com/NLPrinceton/sparse_recovery/master/solvers.py'
wget.download(file1_path, 'solvers.py')
from solvers import *

In [3]:
## Define CKSAAP feature-extraction function
def minSequenceLength(fastas):
	minLen = 10000
	for i in fastas:
		if minLen > len(i[1]):
			minLen = len(i[1])
	return minLen

def CKSAAP(fastas, gap=5, **kw):
	if gap < 0:
		print('Error: the gap should be equal or greater than zero' + '\n\n')
		return 0

	if minSequenceLength(fastas) < gap+2:
		print('Error: all the sequence length should be larger than the (gap value) + 2 = ' + str(gap+2) + '\n\n')
		return 0

	AA = 'ACDEFGHIKLMNPQRSTVWY'
	encodings = []
	aaPairs = []
	for aa1 in AA:
		for aa2 in AA:
			aaPairs.append(aa1 + aa2)
	header = ['#']
	for g in range(gap+1):
		for aa in aaPairs:
			header.append(aa + '.gap' + str(g))
	encodings.append(header)
	for i in fastas:
		name, sequence = i[0], i[1]
		code = [name]
		for g in range(gap+1):
			myDict = {}
			for pair in aaPairs:
				myDict[pair] = 0
			sum = 0
			for index1 in range(len(sequence)):
				index2 = index1 + g + 1
				if index1 < len(sequence) and index2 < len(sequence) and sequence[index1] in AA and sequence[index2] in AA:
					myDict[sequence[index1] + sequence[index2]] = myDict[sequence[index1] + sequence[index2]] + 1
					sum = sum + 1
			for pair in aaPairs:
				code.append(myDict[pair] / sum)
		encodings.append(code)
	return encodings

In [4]:
def delta_rule(A,delta_y,x,b):
  # num_samples_per_class = int(x.shape[0]/2)
  delta1 = 0*x
  delta2 = 0*x
  # delta1[0:num_samples_per_class] = x[0:num_samples_per_class]
  # delta2[num_samples_per_class:] = x[num_samples_per_class:]

  delta1[delta_y==1]=x[delta_y==1]
  delta2[delta_y==0]=x[delta_y==0]

  y1 = np.matmul(A,delta1)
  y2 = np.matmul(A,delta2)
  # print(delta1.shape, delta2.shape, y1.shape, y2.shape)
  r1 = np.linalg.norm(y1-b)
  r2 = np.linalg.norm(y2-b)

  if(r1<r2):
    label = 1
  else:
    label = 0

  return label


In [5]:
def yoden_index(y, y_pred):
  epsilon = 1e-30
  tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
  j = (tp/(tp + fn + epsilon)) + (tn/(tn+fp + epsilon)) - 1
  return j

def pmeasure(y, y_pred):
    epsilon = 1e-30
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    sensitivity = tp / (tp + fn + epsilon)
    specificity = tn / (tn + fp + epsilon)
    f1score = (2 * tp) / (2 * tp + fp + fn + epsilon)
    return ({'Sensitivity': sensitivity, 'Specificity': specificity, 'F1-Score': f1score})

In [6]:
def Calculate_Stats(y_actual,y_pred):
  acc = accuracy_score(y_actual, y_pred)
  sen = pmeasure(y_actual, y_pred)['Sensitivity']
  spe = pmeasure(y_actual, y_pred)['Specificity']
  f1 = pmeasure(y_actual, y_pred)['F1-Score']
  mcc = matthews_corrcoef(y_actual, y_pred)
  bacc = balanced_accuracy_score(y_actual, y_pred)
  yi = yoden_index(y_actual, y_pred)
  #auc = roc_auc_score(y_actual, y_pred)
  
  #pre, rec, _ = precision_recall_curve(y_actual, y_score, pos_label=1)
  #fpr, tpr, _ = roc_curve(y_actual, y_score, pos_label=1)
  #auroc = auc(fpr, tpr)
  #aupr = auc(rec, pre)

  return acc, sen, spe, f1, mcc, bacc, yi

In [7]:
train_set = pd.read_csv("https://raw.githubusercontent.com/Shujaat123/AFP-SRC/master/data/train1.csv")
test_set = pd.read_csv("https://raw.githubusercontent.com/Shujaat123/AFP-SRC/master/data/test1.csv")

In [13]:
X_train = train_set.iloc[:, 1:].to_numpy()
y_train = np.asarray(train_set.CLASS)
y_train[y_train=='AFP']=1
y_train[y_train=='NON_AFP']=0
# y_train = to_categorical(y_train)

X_test = test_set.iloc[:, 1:].to_numpy()
y_test = np.asarray(test_set.CLASS)
y_test[y_test=='AFP']=1
y_test[y_test=='NON_AFP']=0
# y_test = to_categorical(y_test)

# print(X_train.shape,X_test.shape)
# print(y_train.shape,y_test.shape)

X = np.concatenate((X_train,X_test),axis = 0)
y = np.concatenate((y_train,y_test),axis = 0)

print(X.shape, y.shape)

(9972, 840) (9972,)


In [58]:
def custom_train_test_split(X,y,train_sample):
  from random import sample
  n_pos = sum(y==1)
  n_neg = sum(y==0)
  print(n_pos,n_neg)

  pos_list = range(0,n_pos)
  neg_list = range(0,n_neg)

  train_pos_list = sample(pos_list,train_sample)
  train_neg_list = sample(neg_list,train_sample)

  test_pos_list = list(set(list(pos_list)) - set(train_pos_list))
  test_neg_list = list(set(list(neg_list)) - set(train_neg_list))

  pos_sample = X[y==1]
  pos_label = y[y==1]

  neg_sample = X[y==0]
  neg_label = y[y==0]

  X_train = np.concatenate((pos_sample[train_pos_list],neg_sample[train_neg_list]), axis=0)
  y_train = np.concatenate((pos_label[train_pos_list],neg_label[train_neg_list]), axis=0)

  X_test = np.concatenate((pos_sample[test_pos_list],neg_sample[test_neg_list]), axis=0)
  y_test = np.concatenate((pos_label[test_pos_list],neg_label[test_neg_list]), axis=0)

  print(X_train.shape, y_train.shape)
  print(X_test.shape, y_test.shape)

  return X_train, X_test, y_train, y_test


In [59]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.8, random_state=0)
X_train, X_test, y_train, y_test = custom_train_test_split(X,y,300)

from sklearn.decomposition import KernelPCA
# transformer = KernelPCA(n_components=100, kernel='linear')
transformer = KernelPCA(n_components=100, kernel='poly') # 'linear', 'poly', 'rbf', ‘sigmoid’, ‘cosine’
transformer.fit_transform(X_train)
X_train = transformer.transform(X_train)
X_test = transformer.transform(X_test)

X_train = np.transpose(X_train)
y_train = np.transpose(y_train).astype(int)
X_test = np.transpose(X_test)
y_test = np.transpose(y_test).astype(int)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

481 9491
(600, 840) (600,)
(9372, 840) (9372,)
(100, 600) (100, 9372)
(600,) (9372,)


In [60]:
def Test_SRC(A,delta_y,DATA,LABEL,verbose):
  # A = X_train
  # DATA = X_test
  # LABEL = y_test
  LABEL_PRED = []
  count = 0
  for ind in range(0,DATA.shape[1]):
    b = DATA[:,ind]
    x = NonnegativeBP(A, b, x0=None, tol=1E-4, niter=100, biter=32)
    label_out = delta_rule(A,delta_y,x,b)
    if (verbose):
      check = label_out==LABEL[ind]
      if (check):
        count = count + 1
      accuracy = 100*count/(ind+1)
      print(ind+1, count, accuracy, LABEL[ind], label_out, check)
    LABEL_PRED.append(label_out)

  return np.array(LABEL_PRED)


In [61]:
# y_train_pred = Test_SRC(X_train,y_train,X_train,y_train,0)
y_test_pred = Test_SRC(X_train,y_train,X_test,y_test,1)

Streaming output truncated to the last 5000 lines.
4373 3741 85.54767893894352 0 0 True
4374 3741 85.5281207133059 0 1 False
4375 3742 85.53142857142858 0 0 True
4376 3743 85.53473491773309 0 0 True
4377 3744 85.53803975325566 0 0 True
4378 3744 85.51850159890361 0 1 False
4379 3745 85.52180863210779 0 0 True
4380 3746 85.52511415525115 0 0 True
4381 3747 85.52841816936773 0 0 True
4382 3748 85.53172067549065 0 0 True
4383 3749 85.53502167465207 0 0 True
4384 3750 85.53832116788321 0 0 True
4385 3751 85.54161915621437 0 0 True
4386 3752 85.54491564067487 0 0 True
4387 3753 85.54821062229314 0 0 True
4388 3754 85.55150410209663 0 0 True
4389 3755 85.55479608111187 0 0 True
4390 3756 85.55808656036446 0 0 True
4391 3757 85.56137554087907 0 0 True
4392 3758 85.56466302367942 0 0 True
4393 3758 85.54518552242203 0 1 False
4394 3758 85.52571688666363 0 1 False
4395 3759 85.52901023890784 0 0 True
4396 3760 85.53230209281165 0 0 True
4397 3761 85.53559244939731 0 0 True
4398 3762 85.53888130

In [62]:
tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi = Calculate_Stats(y_train, y_train_pred)
t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi = Calculate_Stats(y_test,y_test_pred)

ValueError: ignored

In [20]:
print(tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi)
print(t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi)


1.0 1.0 1.0 1.0 1.0 1.0 1.0
0.9547632311977716 0.06235565819861432 1.0 0.11739130434782609 0.24398028244560158 0.5311778290993071 0.06235565819861422


In [124]:
# Stats=[]

# for i in range(3):
#   y_train_pred = train_list[i]
#   y_test_pred = test_list[i]
  
#   ## Training Measures
#   tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi = Calculate_Stats(y_train, y_train_pred);
  
#   ## Validation Measures
#   #v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi = Calculate_Stats(to_categorical(y_val),y_val_pred);
  
#   ## Test Measures
#   t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi = Calculate_Stats(y_test,y_test_pred);

#   Stats.append([tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi,
#                 #              v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi,
#                 t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi])

# Statistics = np.asarray(Stats)

NameError: ignored

In [122]:
# def Show_Statistics(msg,mean_Stats, sd_Stats, sigfig):
#   print(msg.upper())
#   print(70*'-')
#   print('Accuracy:{} + {}'          .format(round(mean_Stats[0],sigfig), round(sd_Stats[0],sigfig)))
#   print('Sensitivity:{} + {} '      .format(round(mean_Stats[1],sigfig), round(sd_Stats[1],sigfig)))
#   print('Specificity:{} + {}'       .format(round(mean_Stats[2],sigfig), round(sd_Stats[2],sigfig)))
#   print('F1-Score:{} + {}'          .format(round(mean_Stats[3],sigfig), round(sd_Stats[3],sigfig)))
#   print('MCC:{} + {}'               .format(round(mean_Stats[4],sigfig), round(sd_Stats[4],sigfig)))
#   print('Balance Accuracy:{} + {}'  .format(round(mean_Stats[5],sigfig), round(sd_Stats[5],sigfig)))
#   print('Youden-Index:{} + {}'      .format(round(mean_Stats[6],sigfig), round(sd_Stats[6],sigfig)))
#   print(70*'-')

In [123]:
# Show_Statistics('Norm Training Results (MEAN)',Statistics[0][0:7],Statistics.std(axis=0)[0:7], 3)
# Show_Statistics('Norm Test Results (MEAN)',Statistics[0][7:14],Statistics.std(axis=0)[7:14], 3)
# Show_Statistics('Dict Training Results (MEAN)',Statistics[1][0:7],Statistics.std(axis=0)[0:7], 3)
# Show_Statistics('Dict Test Results (MEAN)',Statistics[1][7:14],Statistics.std(axis=0)[7:14], 3)
# Show_Statistics('Rec Training Results (MEAN)',Statistics[2][0:7],Statistics.std(axis=0)[0:7], 3)
# Show_Statistics('Rec Test Results (MEAN)',Statistics[2][7:14],Statistics.std(axis=0)[7:14], 3)
# #Show_Statistics('Test Results (MEAN)',Statistics.mean(axis=0)[14:21],Statistics.std(axis=0)[14:21], 3)

NameError: ignored